In [71]:
import numpy as np
from sympy import symbols
from sympy.parsing.sympy_parser import parse_expr
from sys import exit, stderr
from inspect import currentframe
def debug(*args):
    names = {id(v):k for k,v in currentframe().f_back.f_locals.items()}
    print(', '.join(names.get(id(arg),'???')+' = '+repr(arg) for arg in args), file=stderr)
    return


In [98]:
A = np.array([[1, 2, 7], [2, 3, 1], [3, 1, 2]])
np.linalg.eig(A)

(array([ 7.19615242, -3.19615242,  2.        ]),
 array([[-0.72430711, -0.86422135,  0.30151134],
        [-0.46626671,  0.20478768, -0.90453403],
        [-0.50791197,  0.45954702,  0.30151134]]))

In [103]:
# Power method
x = np.transpose(np.array([1,1,1]))
for i in range(10):
    r = (A@x)[0]/x[0]
    print(r)
    x = A@x

10.0
6.4
7.625
7.024590163934426
7.276546091015169
7.161481719050673
7.211805011307911
7.189265133262127
7.199228145064849
7.194790771570552


In [49]:
# jacobi method A: symmetrical matrix, N: dimension of A

def jacobi_method(A, N):
    global EPS
    # 初期化
    B = A
    U = np.eye(N)
    OK = False
    while(not OK):
        # p, qを選ぶ
        v = 0
        for i in range(N):
            for j in range(i+1, N):
                if i == j:
                    continue
                elif v < abs(B[i][j]):
                    p, q = i, j
                    v = abs(B[i][j])
        # 回転行列を作る
        rot = np.eye(N)
        if B[q][q]-B[p][p] == 0:
            theta = np.pi/4  
        else:
            theta = 1/2*np.arctan(2*B[p][q]/(B[q][q]-B[p][p]))
#         print(theta)
        rot[p][p] = rot[q][q]  = np.cos(theta)
        rot[q][p] = -np.sin(theta)
        rot[p][q] = np.sin(theta)
#         print(rot)
        # 左から逆、右から普通にかける
        B = np.transpose(rot)@B@rot

        # 固有値の更新
        U = U@rot

        # 対角行列チェック, 誤差つぶし
        OK = True
        for i in range(N):
            for j in range(i+1,N):
                if i == j:
                    continue
                elif abs(B[i][j]) < EPS:
                    B[i][j] = 0
                    B[j][i] = 0
                else:
                    OK = False
                    break
            if not OK:
                break
#         print(B)
        if OK:
            return (B, U)

In [50]:
A = np.array([[1, 1/2, -(3**0.5)/2], [1/2, 4, (3**0.5)], [-(3**0.5)/2, (3**0.5), 2]])
EPS = 10e-9
N = 3
B, U = jacobi_method(A, N)
print(B)
print(U)

[[2.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 5.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.34015774e-16]]
[[ 0.70710678  0.          0.70710678]
 [ 0.35355339  0.8660254  -0.35355339]
 [-0.61237244  0.5         0.61237244]]


In [67]:
# Householder 
N = 4
A0 = np.array([[6, 4, 1, 1], [4, 6, 1, 1], [1, 1, 5, 2], [1, 1, 2, 5]])
A = A0
A

array([[6, 4, 1, 1],
       [4, 6, 1, 1],
       [1, 1, 5, 2],
       [1, 1, 2, 5]])

In [68]:
def householder(A, N):
    global EPS
    for i in range(N-2):
        # σの計算
        sig = np.sqrt(sum([a**2 for a in A[i][i+1:]]))
        debug(sig)

        # v -> u , u-> Pの計算
        v = [0 for _ in range(i+1)]
        v += [a for a in A[i][i+1:]]
        # 正なら正を足す、　負ならひく
        v[i+1] = v[i+1] + sig if v[i+1] > 0 else v[i+1]-sig
        v = np.array(v, ndmin=2)
        v = np.reshape(v, (N,1))
        debug(v)
        u = v / np.linalg.norm(v)
        debug(u)
        P = np.eye(N) - 2*u@np.transpose(u)
        debug(P)
        p = 2*(A@u)
        # K = ut@p, q = p - K@u
        K = np.transpose(u)@p
        debug(K)
        q = p - K*u
        debug(q)
        A = A-q@np.transpose(u)-u@np.transpose(q)
    for i in range(N):
        for j in range(N):
            if abs(A[i][j]) < EPS:
                A[i][j] = 0
    return A

In [69]:
householder(A, N)

sig = 4.242640687119285
v = array([[0.        ],
       [8.24264069],
       [1.        ],
       [1.        ]])
u = array([[0.        ],
       [0.98559856],
       [0.11957316],
       [0.11957316]])
P = array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.94280904, -0.23570226, -0.23570226],
       [ 0.        , -0.23570226,  0.97140452, -0.02859548],
       [ 0.        , -0.23570226, -0.02859548,  0.97140452]])
K = array([[13.]])
q = array([[ 8.3630811 ],
       [-0.50730594],
       [ 2.09077028],
       [ 2.09077028]])
sig = 1.4142135623730954
v = array([[ 0.        ],
       [ 0.        ],
       [-2.41421356],
       [-1.        ]])
u = array([[ 0.        ],
       [ 0.        ],
       [-0.92387953],
       [-0.38268343]])
P = array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.70710678, -0.70710678],
       [ 0.        ,  0.        , -0.70

array([[ 6.        , -4.24264069,  0.        ,  0.        ],
       [-4.24264069,  7.        ,  1.41421356,  0.        ],
       [ 0.        ,  1.41421356,  6.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  3.        ]])

In [78]:
A = np.array([[5, 6, 8], [6, 4, 2], [8,2,1]])
N = 3
R = householder(A, N)
R

sig = 10.0
v = array([[ 0.],
       [16.],
       [ 8.]])
u = array([[0.        ],
       [0.89442719],
       [0.4472136 ]])
P = array([[ 1. ,  0. ,  0. ],
       [ 0. , -0.6, -0.8],
       [ 0. , -0.8,  0.6]])
K = array([[10.]])
q = array([[17.88854382],
       [ 0.        ],
       [ 0.        ]])


array([[  5., -10.,   0.],
       [-10.,   4.,   2.],
       [  0.,   2.,   1.]])

In [87]:
print(R[:-2,:-2])
len(R[:-2,:-2])

[[5.]]


1

In [96]:
# 固有多項式の更新
l = symbols('l')
def eigen_polynomial(i):
    if i == 0:
        return 1
    elif i == 1:
        return l-R[0][0]
    else:
        return (l-R[i-1][i-1])*eigen_polynomial(i-1) - (R[i-1][i-2]**2)*eigen_polynomial(i-2)
eigen_polynomial(3).expand()

l**3 - 10.0*l**2 - 75.0*l + 100.0